In [5]:
## 목적: PDF 파일들 => vectorstore (faise_index)
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [4]:
## 1) 문서 업로드: pdf 읽고 doc_list 저장

data_dir = "/Users/hyungseok/Desktop/src/history_chatbot/data"

doc_list = []
pdf_list = sorted([i for i in os.listdir(data_dir) if i.endswith(".pdf")])

for idx, pdf_name in enumerate(pdf_list):
    loader = PyPDFLoader(os.path.join(data_dir, pdf_name))
    documents = loader.load_and_split()
    doc_list.extend(documents)
    print(f"{idx+1}/{len(pdf_list)} {pdf_name} loaded")
    
print(f"Total {idx+1} documents loaded")

1/22 5-2-1-1(1지도서).pdf loaded
2/22 5-2-1-1(2지도서).pdf loaded
3/22 5-2-1-1(3지도서).pdf loaded
4/22 5-2-1-1(4지도서).pdf loaded
5/22 5-2-1-1(5지도서).pdf loaded
6/22 5-2-1-1(6지도서).pdf loaded
7/22 5-2-1-1(7지도서).pdf loaded
8/22 5-2-1-2(10지도서).pdf loaded
9/22 5-2-1-2(11지도서).pdf loaded
10/22 5-2-1-2(12지도서).pdf loaded
11/22 5-2-1-2(13지도서).pdf loaded
12/22 5-2-1-2(14지도서).pdf loaded
13/22 5-2-1-2(8지도서).pdf loaded
14/22 5-2-1-2(9지도서).pdf loaded
15/22 5-2-1-3(15지도서).pdf loaded
16/22 5-2-1-3(16지도서).pdf loaded
17/22 5-2-1-3(17지도서).pdf loaded
18/22 5-2-1-3(18지도서).pdf loaded
19/22 5-2-1-3(19-20지도서).pdf loaded
20/22 5-2-1-3(21지도서).pdf loaded
21/22 5-2-1-3(22지도서).pdf loaded
22/22 5-2-1-3(23-24지도서).pdf loaded
Total 22 documents loaded


In [7]:
## 2) 문서 분할
import tiktoken

def tiktoken_len(text):
    tokenizer = tiktoken.get_encoding("cl100k_base")
    tokens = tokenizer.encode(text)
    return len(tokens)

text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=900,
        chunk_overlap=100,
        length_function=tiktoken_len
    )

chunks = text_splitter.split_documents(doc_list)

In [9]:
## 3) 문서 임베딩
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="jhgan/ko-sroberta-multitask",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)  

vectordb = FAISS.from_documents(chunks, embeddings)
vectordb.save_local("faiss_index")